In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os.path
import pretty_midi
import pickle
import random

### Evaluate system performance

In [3]:
def readGroundTruthLabels(gtfile):
    d={}
    d1={}
    with open(gtfile, "r") as f:
        for line in f:
            data = line.split(',')
            key = data[0]
            d[key] = []
            d1[key] = key
            for idx,item in enumerate(data):
                if idx != 0 and item != "x":
                    try:
                        pieceNum = int(item)
                        d[key].append(pieceNum)
                    except:
                        pass
                    try:
                        pieceNum = int(item)
                        d1[str(pieceNum)] = key
                    except:
                        pass
    return d, d1

In [4]:
def readHypothesisFiles(hypdir, benchmark):
    l = []
    if benchmark == 1:
        for hypfile in sorted(glob.glob("{}/*.pkl".format(hypdir))):
            with open(hypfile, "rb") as f:
                l.append([os.path.splitext(os.path.basename(hypfile))[0],pickle.load(f)])
    elif benchmark == 0 or benchmark == 2:
        for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
            # print(hypfile)
            with open(hypfile, "rb") as f:
                l.append(pickle.load(f))
    elif benchmark == 3:
        for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
            print(hypfile)
            with open(hypfile, "rb") as f:
                data = pickle.load(f)
                pieceScores = [(x[1],x[0]) for x in data[1]]
                l.append((data[0],pieceScores))
    return l

In [5]:
s = set([])
def collapseIds(pieceScores):
    pieceScores = list(dict.fromkeys(pieceScores))
    return pieceScores
        

In [6]:
def getRank(pieceScores,gt,idt,queryid,benchmark, condition):
    query = queryid.split('_')[0]
    l = []
    count = 0
    if benchmark:
        pieceScores = collapseIds(pieceScores)
    rank = 300000
    pklfile = 'piece_to_num.pkl'
    with open (pklfile,'rb')as f:
        piece_to_num = pickle.load(f)
    for i in range(len(pieceScores)):
        if benchmark == 1 or benchmark == 2:
            num = pieceScores[i].split("_")[0]
            if 'p' in num:
                if condition == 2:
                    continue
                if query.split("_")[0] == num:
                    rank = count+1
                    return rank
            elif int(num) in gt[query]:
                rank = count+1
                return rank
            count = count+1            
        else:
            pieceName = piece_to_num[pieceScores[i][0]]
            if pieceName[0]=='p':
                if condition == 2:
                        continue
                if(pieceName==queryid.split('_')[0]):
                    rank=count+1
                    return rank
                if not pieceName in idt.keys():
                    count+=1
                elif not idt[pieceName] in l:
                    count+=1
                    l.append(idt[idt[pieceName]])
            else:
                try:
                    if (int(pieceName.split("**")[-1]) in gt[query]):
                        rank = count+1
                        return rank

                    if not str(int(pieceName.split("**")[-1])) in idt.keys():
                        count+=1
                    elif not idt[str(int(pieceName.split("**")[-1]))] in l:
                        count+=1
                        l.append(idt[str(int(pieceName.split("**")[-1]))])
                except:
                    pass
    return rank

In [11]:
def calcPrecisionRecall(hypdir, gtfile, benchmark = False, condition = 1):
    hyps = readHypothesisFiles(hypdir, benchmark)
    gt,idt = readGroundTruthLabels(gtfile)
    MRR = 0
    runtimes = []
    MRRs = []
    if condition == 2:
        valid = []
        with open('condition2.txt','r') as f:
            for line in f:
                valid.append(line.strip())
                
    hyps = sorted(hyps, key = lambda x: int(x[0].split("_")[0][1:])*100+int(x[0].split("_")[1][1:]))
    count = 0
    if np.shape(hyps)[0] == 2:
        for queryid, pieceScores in hyps:
            if condition == 2:
                if not queryid.split('_')[0].strip() in valid:
                    count+=1
                    continue
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            if rank > 5:
                print(queryid)
            MRRs.append((queryid,rank))
            MRR=MRR+1/(rank)
    else:
        for queryid, pieceScores, runtime in hyps:
            if condition == 2:
                if not queryid.split('_')[0].strip() in valid:
                    count+=1
                    continue
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            if rank>5:
                print(queryid)
            MRRs.append((queryid,rank))
            runtimes.append(runtime)
            MRR=MRR+1/(rank)
    
    return MRR/len(MRRs),MRRs, runtimes

In [8]:
def calcOverlap(seg1, seg2):
    overlap_lb = max(seg1[0], seg2[0])
    overlap_ub = min(seg1[1], seg2[1])
    overlap = np.clip(overlap_ub - overlap_lb, 0, None)
    return overlap    

In [13]:
hyps = readHypothesisFiles('experiments/Analysis(10k)',0)

In [12]:
len(hyps[0])

3

In [15]:
with open("piece_to_num.pkl","rb")as f:
    piece_to_num = pickle.load(f)

In [20]:
count = 0
for i in range(2000):
    if hyps[i][0]=="p19_q9":
        count = i
        break
print(hyps[count][0])
pieceScores  =hyps[count][1]
for i in range(20):
    print(piece_to_num[pieceScores[i][0]],pieceScores[i][1])

p19_q9
dLiszt,_FranzTechnische_Studien,_S.146_518119 10
dLiszt,_FranzTechnische_Studien,_S.146_241441 10
dFranck,_C%C3%A9sarBallade,_Op.9_03232 10
dKogel,_Gustav_FriedrichWalkyre-Fantasie_271323 9
dKienzl,_WilhelmBilder_aus_dem_Volksleben,_Op.52_469234 8
dBeethoven,_Ludwig_vanPiano_Sonata_No.32,_Op.111_02023 8
dRheinberger,_Josef_GabrielPiano_Sonata_No.2,_Op.99_207493 7
dLiszt,_FranzWeihnachtsbaum,_S.186_04023 7
dGouin,_PierreWeihnachtsbaum,_S.186_04023 7
dGouin,_PierreWeihnachtsbaum,_S.186_58249 7
dLiszt,_FranzWeihnachtsbaum,_S.186_58249 7
dCzerny,_CarlDie_Vorschule_der_Fingerfertigkeit,_Op.636_267068 6
dDubois,_Th%C3%A9odorePo%C3%A8mes_Virgiliens_30462 6
dJohnson,_BernardElfentanz_210988 6
dKrasuski,_StefanLa_Reine_du_coeur,_Op.41_445830 6
dCapponi,_FrancescoBiblioteca_d%27oro_18486 5
dOrefice,_Giuseppe_dell%27Biblioteca_d%27oro_18486 5
dMariani,_AngeloBiblioteca_d%27oro_18486 5
dRubinstein,_AntonPiano_Sonata_No.1,_Op.12_03371 5
dCzerny,_CarlPianoforte-Schule,_Op.500_356508 5


In [12]:
hypdir = 'experiments/Analysis(10k)'
condition2 = "condition2.txt"

#0 is normal
#1 is old paper
#2 is new paper
#3 is weird test
benchmark = 0
queryGTFile = 'piece_To_id.csv'
MRR, MRRs, runtimes = calcPrecisionRecall(hypdir, queryGTFile, benchmark, condition = 1)

/home/ljh/miniconda3/envs/pytorch/lib/python3.7/site-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


p5_q4
p5_q10
p12_q10
p14_q3
p14_q5
p17_q9
p19_q9
p19_q10
p20_q8
p20_q10
p28_q2
p28_q4
p29_q6
p33_q10
p39_q3
p49_q1
p49_q3
p49_q4
p49_q6
p49_q10
p50_q5
p50_q6
p51_q10
p52_q1
p58_q2
p58_q6
p58_q7
p58_q9
p62_q6
p63_q1
p63_q2
p63_q5
p63_q6
p63_q7
p63_q10
p64_q2
p64_q3
p64_q10
p71_q1
p71_q9
p72_q10
p73_q1
p73_q4
p73_q5
p73_q7
p74_q3
p74_q4
p74_q5
p74_q7
p74_q9
p74_q10
p75_q1
p76_q1
p76_q2
p76_q3
p76_q4
p76_q5
p76_q6
p76_q7
p76_q8
p76_q9
p76_q10
p79_q9
p79_q10
p85_q3
p85_q4
p85_q9
p96_q9
p100_q9
p101_q3
p103_q10
p104_q6
p104_q7
p111_q1
p111_q2
p111_q3
p111_q4
p111_q5
p111_q6
p111_q7
p111_q8
p111_q9
p111_q10
p113_q6
p115_q5
p115_q6
p115_q10
p118_q3
p118_q5
p118_q7
p118_q9
p118_q10
p119_q10
p120_q8
p120_q10
p124_q8
p130_q3
p133_q10
p135_q3
p135_q4
p136_q4
p139_q8
p139_q9
p139_q10
p142_q6
p144_q3
p144_q6
p148_q6
p148_q10
p149_q2
p150_q3
p152_q4
p153_q10
p154_q1
p154_q4
p154_q6
p154_q9
p154_q10
p155_q7
p156_q4
p159_q1
p159_q2
p159_q5
p159_q7
p159_q9
p159_q10
p160_q2
p160_q4
p160_q7
p161_q1
p164_

In [ ]:
hypdir = 'experiments/Analysis(10k)'
hyps = readHypothesisFiles(hypdir, 0)
longtime_list = []
for queryid, pieceScores, runtime in hyps:
    if runtime>4:
        print(queryid)
        longtime_list.append(queryid)

In [26]:
print(longtime_list)

['p109_q10', 'p109_q5', 'p109_q6', 'p10_q4', 'p11_q10', 'p11_q2', 'p172_q6', 'p178_q8', 'p185_q6', 'p186_q4', 'p186_q5', 'p186_q6', 'p18_q4', 'p18_q5', 'p198_q5', 'p26_q3', 'p26_q4', 'p26_q8', 'p26_q9', 'p51_q8', 'p51_q9', 'p58_q1', 'p58_q10', 'p58_q2', 'p58_q3', 'p58_q4', 'p65_q4', 'p7_q5', 'p7_q6', 'p7_q7', 'p90_q4', 'p90_q8', 'p92_q2', 'p92_q3', 'p93_q1']


In [33]:
for i in range(len(longtime_list)):
    longtime_list[i] = "data/queries/"+longtime_list[i]+".jpg"

In [19]:
inputs = []
query_list = "cfg_files/query.part.list"
with open(query_list, 'r') as f:
    for line in f:
        inputs.append(line.rstrip())

In [20]:
print(inputs[0])

data/queries/p1_q1.jpg


In [21]:
from ExtractBootlegFeatures import *
for query in inputs:
    bs,notes_num,runtime = processQuery(query)
    results.append((notes_num,runtime))

Processing data/queries/p1_q1.jpg
Processing data/queries/p1_q2.jpg
Processing data/queries/p1_q3.jpg
Processing data/queries/p1_q4.jpg
Processing data/queries/p1_q5.jpg
Processing data/queries/p1_q6.jpg
Processing data/queries/p1_q7.jpg
Processing data/queries/p1_q8.jpg
Processing data/queries/p1_q9.jpg
Processing data/queries/p1_q10.jpg
Processing data/queries/p2_q1.jpg
Processing data/queries/p2_q2.jpg


In [18]:
len(results)

453

In [2]:
fin_results = []
flag = 0
for i in range(len(results)):
    if 
    x = results[i][0]
    if x!=flag:
        fin_results.append(results[i])
        flag = x
    else:
        for j in range(i+1,len(results)):
            x = results[j][0]
            if x == flag:
                continue
            else:
                amount = j-i
                if amount%2==0:
                    y1 = results[i+amount/2-1][1]
                    y2 = results[i+amount/2][1]
                    y = (y1+y2)/2
                else:
                    y = results[i+amount//2][1]
                fin_results.append()
                break
        


Processing data/queries/p1_q6.jpg
(232, 1.0943620204925537)


Processing data/queries/p155_q8.jpg


231

In [6]:
len(results)

1546

In [5]:
np.mean(results)

231.839

In [37]:
print(results)

[721, 657, 624, 354, 459, 394, 518, 539, 374, 606, 747, 722, 392, 486, 156, 512, 580, 642, 585, 352, 388, 583, 540, 649, 673, 546, 661, 382, 423, 500, 574, 497, 600, 547, 156]


In [36]:
np.mean(results)

518.2571428571429

In [10]:
MRR

0.9047623900012717

In [11]:
len(runtimes)

2000

In [12]:
a = np.array(runtimes)
print(np.mean(a),np.std(a))

2.0537587550878524 0.8179777061124105


In [16]:
from ExtractBootlegFeatures import *


In [21]:
imagefile = "data/queries/p65_q4.jpg"
bs = processQuery(imagefile)
len(bs[0])

Processing data/queries/p65_q4.jpg


152

In [ ]:
keys = list(idt.keys())
print(keys)

In [19]:
MRR

0.9015274480131495

In [25]:
getAverage(runtimes)

2.03541009709239

### Investigate Errors

In [11]:
def printDebuggingInfo(MRRs):
    for i, (queryid, rank) in enumerate(MRRs):
        print(queryid, rank)

In [12]:
printDebuggingInfo(MRRs)

p2_q1 1
p2_q2 1
p2_q3 1
p2_q4 1
p2_q5 1
p2_q6 1
p2_q7 1
p2_q8 1
p2_q9 1
p2_q10 1
p3_q1 1
p3_q2 1
p3_q3 1
p3_q4 1
p3_q5 1
p3_q6 1
p3_q7 1
p3_q8 1
p3_q9 1
p3_q10 1
p4_q1 1
p4_q2 1
p4_q3 1
p4_q4 1
p4_q5 1
p4_q6 1
p4_q7 1
p4_q8 1
p4_q9 1
p4_q10 1
p6_q1 1
p6_q2 1
p6_q3 1
p6_q4 1
p6_q5 1
p6_q6 1
p6_q7 1
p6_q8 1
p6_q9 1
p6_q10 1
p7_q1 1
p7_q2 1
p7_q3 1
p7_q4 1
p7_q5 1
p7_q6 1
p7_q7 1
p7_q8 1
p7_q9 1
p7_q10 1
p8_q1 1
p8_q2 1
p8_q3 1
p8_q4 1
p8_q5 1
p8_q6 1
p8_q7 1
p8_q8 1
p8_q9 1
p8_q10 1
p9_q1 1
p9_q2 1
p9_q3 1
p9_q4 1
p9_q5 1
p9_q6 1
p9_q7 1
p9_q8 1
p9_q9 1
p9_q10 1
p10_q1 1
p10_q2 1
p10_q3 1
p10_q4 1
p10_q5 1
p10_q6 1
p10_q7 1
p10_q8 1
p10_q9 1
p10_q10 1
p12_q1 1
p12_q2 1
p12_q3 1
p12_q4 1
p12_q5 1
p12_q6 1
p12_q7 1
p12_q8 1
p12_q9 1
p12_q10 300000
p13_q1 1
p13_q2 1
p13_q3 1
p13_q4 1
p13_q5 1
p13_q6 1
p13_q7 1
p13_q8 1
p13_q9 1
p13_q10 1
p14_q1 1
p14_q2 1
p14_q3 300000
p14_q4 1
p14_q5 300000
p14_q6 1
p14_q7 2
p14_q8 1
p14_q9 1
p14_q10 1
p16_q1 1
p16_q2 1
p16_q3 1
p16_q4 1
p16_q5 1
p16_q6 1


### Measure Runtime

In [11]:
def showRuntimeStats(durs):
    durs = []
    durs = np.array(durs)
    avgDur = np.mean(durs)
    minDur = np.min(durs)
    maxDur = np.max(durs)
    stdDur = np.std(durs)
    print('Avg Duration: {:.2f} sec'.format(avgDur))
    print('Std Duration: {:.2f} sec'.format(stdDur))
    print('Min Duration: {:.2f} sec'.format(minDur))
    print('Max Duration: {:.2f} sec'.format(maxDur))
    plt.hist(durs, bins=np.arange(0,2,.1))
    plt.xlabel('Runtime (sec)')
    plt.ylabel('Count')

In [11]:
print(np.mean(runtimes))

3.4768030458688735


In [ ]:
showRuntimeStats(runtimes)

In [ ]:
print(sorted(glob.glob("*.jpg")))

In [ ]:
import os
list = ""
dir = 'data/queries'
for i in range(1,201):
    for j in range(1,11):
        list+=dir+'/p'+str(i)+'_q'+str(j)+'.jpg\n'
with open('cfg_files/query.total.list','w')as f:
    f.write(list)

## 新的测试程序

In [ ]:
from Singel_Query import *

In [11]:
def getRecallPrecision(pieceScores,gt,idt,queryid,piece_to_num):
    l = []
    topN = 30
    querygt = gt[queryid.split('_')[0]]
    totalgt = len(querygt)+1
    count = 0
    results = []
    for i in range(topN):
        pieceName = piece_to_num[pieceScores[i][0]]
        if pieceName[0]=='p':
            if pieceName==queryid.split('_')[0]:
                count+=1
        elif int(pieceName.split('_')[-1]) in querygt:
            count+=1
        results.append((count/totalgt,count/(i+1)))
    return results

In [5]:
def calcPrecisionRecall(hypdir, gtfile, benchmark = False):
    hyps = readHypothesisFiles(hypdir, benchmark)
    gt,idt = readGroundTruthLabels(gtfile)
    MRR = 0
    runtimes = []
    recallPrecision = []
    pklfile = 'piece_to_num.pkl'
    with open (pklfile,'rb')as f:
        piece_to_num = pickle.load(f)
                
    hyps = sorted(hyps, key = lambda x: int(x[0].split("_")[0][1:])*100+int(x[0].split("_")[1][1:]))
    count = 0
    for queryid, pieceScores, runtime in hyps:
        results = getRecallPrecision(pieceScores,gt,idt,queryid,piece_to_num)
        recallPrecision.append(results)
        runtimes.append(runtime)
    
    meanrecallPrecision = []
    for j in range(30):
        total_recall = 0
        total_precision = 0
        for i in range(len(recallPrecision)):
            total_recall += recallPrecision[i][j][0]
            total_precision += recallPrecision[i][j][1]
        mean_recall = total_recall/len(recallPrecision)
        mean_precision = total_precision/len(recallPrecision)
        meanrecallPrecision.append((mean_recall,mean_precision))

    return meanrecallPrecision, runtimes

In [12]:
hypdir = 'experiments/Analysis(10k)'
condition2 = "condition2.txt"

#0 is normal
#1 is old paper
#2 is new paper
#3 is weird test
benchmark = 0
queryGTFile = 'piece_To_id.csv'
meanrecallPrecision, runtimes = calcPrecisionRecall(hypdir, queryGTFile, benchmark)

ValueError: invalid literal for int() with base 10: 'K.27956310'

In [8]:
def getAverage(list):
    sum = 0
    for number in list:
        sum+=number
    return sum/len(list)

In [ ]:
testlist = 'cfg_files/query.train.list'
pickle_file = 'experiments/indices/Dynamic_N_GRAM_ALL(2).pkl'
piecenumfile = 'piece_to_num.pkl'

with open(piecenumfile,'rb')as f:
    piece_to_num=pickle.load(f)

with open(pickle_file,'rb')as f:
    rindex = pickle.load(f)



In [ ]:
MRRs = []
Runtimes = []
with open(testlist,'r') as f:
    for curfile in f:
        curfile = curfile.strip().strip('\n')
        pieceScores, runTime =  processSingleQuery(curfile,rindex,"Dynamic_Static")
        grade = 0
        i = 1
        curfile = curfile.split('/')[-1]
        for pieceScore in pieceScores:
            if pieceScore[1] != grade:
                grade = pieceScore[1]
                rank = i
            if piece_to_num[pieceScore[0]]== curfile[0:curfile.index('_')]:
                break
            i += 1
        MRR = 1/rank
        MRRs.append(MRR)
        Runtimes.append(runTime)

print("The MRR of test list is ",getAverage(MRRs))
print("The average running time is ",getAverage(Runtimes))

In [ ]:
i = 0
with open(testlist,'r')as f:
    with open('log(5).list','w')as outfile:
        for curfile in f:
            outfile.write(curfile.strip('\n')+' '+str(MRRs[i])+'\n')
            i+=1

In [ ]:
pieceScores, runTime =  processSingleQuery('data/queries/p2_q1.jpg',rindex,"Dynamic_Static")
rank = 1
grade = 0
i = 1
for pieceScore in pieceScores:
    if pieceScore[1] != grade:
        rank = i
    if piece_to_num[pieceScore[0]]== 'p2':
        break
    i += 1

MRR = 1/rank
print(MRR)
print(len(pieceScores))
for i in range(10):
    print(pieceScores[i])


In [ ]:
list = 'log(5).list'
MRRs = []
with open(list,'r')as f:
    for curfile in f:
        MRRs.append(float(curfile.split()[1]))

print(len(MRRs))
print(getAverage(MRRs))

In [ ]:
path = "experiments/Analysis/hyp"
pklfile = "piece_to_num.pkl"
ranks = []
with open (pklfile,'rb')as f:
    piece_to_num = pickle.load(f)

for home, dirs, files in os.walk(path):
    for file in files:
        with open(os.path.join(home,file),"rb")as f:
            hypfile = pickle.load(f)
            queryid = hypfile[0]
            pieceScores = hypfile[1]
            rank = 1
            grade = 0
            i =1
            for pieceScore in pieceScores:
                # if pieceScore[1] != grade:
                #     rank = i
                # if piece_to_num[pieceScore[0]]== queryid.split('_')[0]:
                #     break
                # i+=1
                pieceName = piece_to_num[pieceScore[0]]
                if pieceName == queryid.split('_')[0]:
                    break
                else:
                    rank+=1
            ranks.append(rank)

MRR = 0
for rank in ranks:
    MRR+=1/rank
MRR = MRR/len(ranks)
print(MRR)


In [ ]:
print(piece_to_num)

In [2]:
with open("experiments/Analysis(20k)/p1_q1.hyp","rb")as f:
    a = pickle.load(f)

In [7]:
a[1]

[(9981, 95),
 (11464, 95),
 (7530, 94),
 (8610, 94),
 (15742, 94),
 (17018, 94),
 (4702, 83),
 (13120, 83),
 (17597, 83),
 (24058, 67),
 (27782, 67),
 (3357, 59),
 (12190, 59),
 (6394, 47),
 (19801, 47),
 (10268, 20),
 (26837, 17),
 (3083, 16),
 (28488, 13),
 (5889, 10),
 (20135, 10),
 (25617, 10),
 (26884, 10),
 (21652, 10),
 (16540, 10),
 (22233, 10),
 (31295, 10),
 (12334, 9),
 (14959, 9),
 (24297, 9),
 (30232, 9),
 (9479, 9),
 (10906, 9),
 (1860, 9),
 (41, 9),
 (5686, 9),
 (31958, 9),
 (22829, 9),
 (26815, 9),
 (16444, 9),
 (5888, 9),
 (705, 8),
 (8167, 8),
 (13917, 8),
 (15212, 8),
 (18563, 8),
 (21641, 8),
 (27159, 8),
 (31502, 8),
 (31646, 8),
 (3870, 8),
 (6514, 8),
 (10178, 8),
 (16137, 8),
 (18114, 8),
 (24747, 8),
 (24888, 8),
 (3250, 8),
 (13525, 7),
 (15943, 7),
 (29074, 7),
 (1646, 7),
 (2638, 7),
 (3074, 7),
 (4985, 7),
 (8074, 7),
 (8617, 7),
 (8721, 7),
 (9763, 7),
 (11120, 7),
 (12851, 7),
 (23239, 7),
 (24883, 7),
 (27687, 7),
 (27864, 7),
 (26982, 7),
 (30299, 7),
 

In [5]:
pklfile = 'piece_to_num.pkl'
with open (pklfile,'rb')as f:
    piece_to_num = pickle.load(f)

In [10]:
for i in range(30):
    print(piece_to_num[a[1][i][0]],a[1][i][1])

dGouin,_PierreNocturnes,_Op.9_34915 95
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_34915 95
dGouin,_PierreNocturnes,_Op.9_112335 94
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_112335 94
dGouin,_PierreNocturnes,_Op.9_34916 94
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_34916 94
p1 83
dGouin,_PierreNocturnes,_Op.9_86550 83
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_86550 83
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_113996 67
dGouin,_PierreNocturnes,_Op.9_113996 67
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_00470 59
dGouin,_PierreNocturnes,_Op.9_00470 59
dGouin,_PierreNocturnes,_Op.9_80717 47
dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_80717 47
dLiszt,_FranzHungarian_Rhapsody_No.2,_S.24433352 20
dLiszt,_FranzHungarian_Rhapsody_No.2,_S.24442480 17
dLiszt,_FranzHungarian_Rhapsody_No.2,_S.24400374 16
dLiszt,_FranzHungarian_Rhapsody_No.2,_S.244300584 13
dKetten,_HenryCaprice_styrien,_Op.107_36651 10
dDvo%C5%99%C3%A1k,_Anton%C3%ADnSilhouettes,_Op.8_09421 10
dClementi,_Muzio2_Piano_Sonatas_and_2_C